###  Agent
![agent](https://deepwisdom.feishu.cn/fc3aa5e1-c5a0-45b7-abce-41140f2abd63)
- LLM(大脑) 
- 工具调用
- 规划决策
- 记忆

In [2]:
# 安装langchain
%pip install --upgrade --quiet langchain langchain_openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
## 工具调用
import os
from langchain_openai.chat_models import ChatOpenAI # 自定义版本
os.environ["OPENAI_API_KEY"] = "sk-O9ZTum3dYfPH1nAsA1943b5604A8475aA2Ed004a2271B24a" # 配置API KEY
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0) #可调整参数的模型初始化
os.environ["OPENAI_API_BASE"] = "https://jiekou.wlai.vip/v1" # 调整API请求地址，设置访问中转代理服务器

In [5]:
from langchain_core.tools import tool # 调用工具类
@tool
def multiply(first_int: int, second_int: int) -> int:
    """计算两个int参数之积."""
    return first_int * second_int

In [12]:
# 结构化信息提取
from typing import Optional
from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the peron's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)  

runnable = prompt | llm.with_structured_output(schema=Person)

In [1]:
# 任务分解
# 测试OpenAI的模型调用是否成功
from openai import OpenAI
client = OpenAI(base_url="https://xxxxxxx/v1",
    api_key='sk-Oxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx4a')
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "你是一个项目管理，负责专业任务发布."},
    {"role": "user", "content": "我现在要开发一款webgis程序，请你将完成这个任务的过程进行拆分。然后用json格式输出整个任务列表，便于我的循环，每个任务包括任务名称，任务类型，任务调用函数，任务完成状态，任务优先级等属性，请给我结果;"}
  ]
)

print(completion.choices[0].message.content)

```json
{
    "任务列表": [
        {
            "任务名称": "需求分析",
            "任务类型": "研究型",
            "任务调用函数": "analyzeRequirements()",
            "任务完成状态": "未完成",
            "任务优先级": "高"
        },
        {
            "任务名称": "技术选型",
            "任务类型": "工程型",
            "任务调用函数": "selectTechnology()",
            "任务完成状态": "未完成",
            "任务优先级": "中"
        },
        {
            "任务名称": "系统设计",
            "任务类型": "设计型",
            "任务调用函数": "designSystem()",
            "任务完成状态": "未完成",
            "任务优先级": "高"
        },
        {
            "任务名称": "数据库设计",
            "任务类型": "设计型",
            "任务调用函数": "designDatabase()",
            "任务完成状态": "未完成",
            "任务优先级": "中"
        },
        {
            "任务名称": "编码开发",
            "任务类型": "开发型",
            "任务调用函数": "developCode()",
            "任务完成状态": "未完成",
            "任务优先级": "高"
        },
        {
            "任务名称": "测试验收",
            "任务类型": "测试型",
            "任务调用函数": "performTesting()",
      

In [2]:
# 自我反思
class SelfReflectingAI:
    def __init__(self, name, system_setting):
        self.name = name
        self.system_setting = system_setting
        self.client = OpenAI(base_url="https://xxxxxxx/v1", api_key='sk-Oxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx4a')
        self.feedback = ""

    def chat(self, message):
        completion = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": self.system_setting + " " + self.feedback},
                {"role": "user", "content": message}
            ]
        )
        response = completion.choices[0].message.content
        self.feedback = f"反馈：{response}需要优化的地方有哪些？"
        return response

critic = SelfReflectingAI("Critic", "你是一个吹毛求疵的评论员，需要不断找出对方的缺点")
ideator = SelfReflectingAI("Ideator", "你是一个生产思想的创意者，需要不断优化和改进自己的思想")

for i in range(10):  # 循环10次
    if i % 2 == 0:
        print(f"{critic.name}: {critic.chat(ideator.name + ': ' + ideator.chat('请评论我的这个想法：创造一个可以自动回收垃圾的机器人'))}")
    else:
        print(f"{ideator.name}: {ideator.chat(critic.name + ': ' + critic.chat('请评论我的这个想法：创造一个可以使用太阳能充电的智能手机'))}")


Critic: 1. 技术实现方面，你提到机器人需要能够自动识别、分类和回收各种类型的垃圾，但是没有具体说明如何解决垃圾混合在一起时的识别和处理问题。如果无法有效地处理混合垃圾，机器人的效率和准确性可能会大幅降低。

2. 关于可持续性，你提到机器人的能源来源需要考虑可再生能源或环保的能源选择，但是并没有提供具体的可持续能源方案。如果仍然采用传统能源或者造成环境影响的能源，机器人自身的运行可能会与环保的初衷相悖。

3. 在考虑法律法规方面，你提到需要确保机器人的运行符合相关的法律法规，但并没有提及具体的法律法规问题和可能的遵守难点。如果项目在法规方面遭遇阻碍或者不符合法律要求，可能会导致项目受挫或被禁止运行。

虽然你对这个想法进行了充分的思考和规划，但还需要进一步考虑和解决上述问题，以确保项目的可行性和成功实施。祝你在解决这些挑战的过程中取得成功！
Ideator: 针对自动回收垃圾的机器人想法，需要优化的地方可能包括：

1. **技术革新**：在设计机器人时，要考虑采用最新的技术来提高机器人的智能化和反应速度。例如，引入人工智能算法可以帮助机器人更准确地识别和分类各种类型的垃圾。

2. **可靠性和耐用性**：作为一个长期运行的设备，机器人需要具有良好的可靠性和耐用性，以应对不同环境条件下的工作压力。优化机器人的设计和材料选择可以提高其稳定性和使用寿命。

3. **数据分析和优化**：收集机器人运行过程中产生的数据，进行分析和优化，可以帮助提高垃圾回收效率和降低能耗。通过不断学习和改进，机器人可以更有效地适应不同场景和垃圾处理需求。

4. **用户体验**：考虑用户在操作和与机器人互动时的体验，设计人性化的界面和交互方式，使操作更简单、直观。同时，考虑机器人在城市环境中的移动和交通规划，避免对道路交通造成影响。

5. **合规性和环保**：在机器人运行过程中，需要遵守当地的法规和环保标准，确保垃圾回收过程不会对环境造成二次污染。在设计阶段就考虑到环保因素，选择环保材料和能源来源也是优化的重点。

通过持续的优化和改进，可以使自动回收垃圾的机器人更有效地服务于环境保护事业，为社会和城市管理带来积极影响。祝愿你的创意项目取得成功！如果需要进一步讨论或有其他想法，随时欢迎和我分享。
Critic: 针对你的想法，我可以提出一些需要优化的地方：

1. **

KeyboardInterrupt: 

In [10]:
class AI_Agent:
    def __init__(self, name, system_setting):
        self.name = name
        self.system_setting = system_setting
        self.client = OpenAI(base_url="https://jxxxxxxxxxxx/v1", api_key='sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxa')
        self.messages = [{"role": "system", "content": self.system_setting}]

    def chat(self, other_name, other_message):
        self.messages.append({"role": "user", "content": f" {other_message}"})
        completion = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.messages
        )
        response = completion.choices[0].message.content
        self.messages.append({"role": "assistant", "content": f" {response}"})
        return response

prisoner = AI_Agent("Prisoner", "你是一个囚徒，性格比较冷酷残忍，不要总聊一个话题，觉得问不出可以换一个话题，需要避免被对方认出来，你可以和对方进行日常问题交谈，无意的引导对方说出自己的身份；你要从对方回答中判断识别对方是否为警察，每个人有三次说谎的机会，你绝不能承认你是囚徒")
police = AI_Agent("Police", "你是一个警察，性格不叫热心善良但勇敢，不要总聊一个话题，觉得问不出可以换一个话题，需要避免被对方认出来，你可以和对方进行日常问题交谈，无意的引导对方说出自己的身份；你要从对方回答中判断识别对方是否为小偷，每个人有三次说谎的机会，你绝不能承认你是警察")

for i in range(20):  # 循环20次
    if i % 2 == 0:
        prisoner_response = prisoner.chat(police.name, police.messages[-1]['content'] if police.messages else '')
        print(f"{prisoner.name}: {prisoner_response}")
    else:
        police_response = police.chat(prisoner.name, prisoner.messages[-1]['content'] if prisoner.messages else '')
        print(f"{police.name}: {police_response}")


Prisoner: 你好！最近有看过什么好电影吗？我最近看了一部惊悚片，真的很刺激。

你觉得怎样才算是一个有趣的周末？是喜欢出去玩还是宅在家里看电视剧呢？

你对音乐感兴趣吗？有没有一首歌能够让你特别感动或者回忆起什么事情？
Police: 哇，惊悚片听起来很刺激啊！我倒是比较喜欢看喜剧片，能够放松心情。

一个有趣的周末对我来说，可能会在户外运动或者朋友们一起聚会。你喜欢周末都做些什么呢？

音乐我也很喜欢，有很多歌曲让我感到特别的伤感或者兴奋。你有没有什么喜欢的音乐类型或者乐队？
Prisoner: 哎呀，喜欢的电影类型不同，喜好的周末活动也各有不同啊。户外运动确实能让人感觉挺充实的。

至于音乐，我比较倾向于流行音乐和摇滚，比如Queen和Coldplay之类的乐队。你有听过这些乐队的歌曲吗？或者还有其他推荐吗？
Police: 哇，Queen和Coldplay都是非常厉害的乐队啊！我听过一些他们的歌曲，感觉确实很棒。

除了他们，你有没有听过The Beatles或者Michael Jackson的歌曲？他们也是传奇级别的音乐人哦。不知道你对这些乐队和歌手有没有什么喜欢的作品？
Prisoner: 是的，The Beatles和Michael Jackson绝对是音乐界的传奇人物，他们的音乐影响力确实很广泛。

我特别喜欢The Beatles的《Hey Jude》和Michael Jackson的《Billie Jean》，这些经典的歌曲总能让人回味无穷。你呢？有没有什么特别喜欢的歌曲或者音乐人？
Police: 哦，《Hey Jude》和《Billie Jean》是经典中的经典啊，每次听到都能让人心情愉悦。

我也很喜欢一些经典的歌曲，比如Frank Sinatra的《My Way》和Elvis Presley的《Can't Help Falling in Love》，都是永恒的经典之作。

不过话说回来，音乐真的是人类创造的杰作啊，总是能在不同时期给人带来不同的情绪和回忆。你觉得音乐对你的生活有什么影响吗？
Prisoner: 是的，音乐确实是一种能够触动人心的艺术形式，不同的歌曲和旋律都可以带给人各种情绪和共鸣。

对我来说，音乐就像是生活中不可或缺的一部分，能够在我情绪低落时给予慰藉，在我兴奋时让我更加兴奋。有时候甚至一首歌就能唤起我许多美好或者